#### This notebook loads historical stock data from the Alpaca API

Load every minute stock data from todays NASDAQ 100 index companies, from 2000-01-01 to 2025-01-01

In [ ]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from pytickersymbols import PyTickerSymbols
from datetime import datetime
import pandas as pd
import os

# Keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

# Create client
stock_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Get all stock symbols in NASDAQ 100
stock_data = PyTickerSymbols()
nasdaq_100 = stock_data.get_stocks_by_index('NASDAQ 100')

# Create empty DataFrame
columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]
stock_df = pd.DataFrame(columns=columns)

stock_nr = 0
for stock in nasdaq_100:
    symbol = stock["symbol"]
    stock_nr += 1
    print(f"Processing stock {stock_nr}/100 - {symbol}")

    # Get the stock bars
    request_params = StockBarsRequest(symbol_or_symbols=[symbol], timeframe=TimeFrame.Minute, start=datetime(2000, 1, 1), end=datetime(2025, 1, 1))
    bars = stock_client.get_stock_bars(request_params)
    
    # Check if the symbol is in the data
    if (symbol in bars.data):
        # Clean stock data and add to DataFrame
        df = pd.DataFrame(bars[stock["symbol"]])
        df = df.map(lambda x: x[1]) 
        df.columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]
        if not df.empty:
            stock_df = pd.concat([stock_df, df], ignore_index=True)

# Save data to a parquet file
stock_df.to_parquet("./data/raw/raw__nasdaq100_2000_to_2025.parquet", engine="pyarrow", index=False)

#### Load every minute data for S&P 500
We use SPY since Alpaca do not provide S&P 500 data through thier API

In [ ]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
import pandas as pd
import os

# Keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

stock_client = StockHistoricalDataClient(API_KEY,  SECRET_KEY)


stock = "SPY"
request_params = StockBarsRequest(symbol_or_symbols=[stock], timeframe=TimeFrame.Minute, start=datetime(2000, 1, 1), end=datetime(2025, 1, 1))
bars = stock_client.get_stock_bars(request_params)

# Flatten the dictionary into a DataFrame
df = pd.DataFrame(bars[stock])

# Extrahera endast värdena från tuple-paren
df = df.map(lambda x: x[1])  # Tar bara det andra elementet i tuple:n

# Sätt kolumnnamn
df.columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]

# Save data to a parquet file
df.to_parquet("./data/raw/raw__sp500_2000_to_2025.parquet", engine="pyarrow", index=False)

#### Raw to Enriched
Add features to data

In [22]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange

def feature_engineering(df, df_sp):
    # Convert timestamp to Unix time (int64) for efficiency
    df['timestamp'] = pd.to_datetime(df['timestamp']).astype('int64') // 10**9
    df_sp['timestamp'] = pd.to_datetime(df_sp['timestamp']).astype('int64') // 10**9
    
    # Sort data by symbol and timestamp
    df = df.sort_values(by=['symbol', 'timestamp'])
    df_sp = df_sp.sort_values(by=['symbol', 'timestamp'])
    
    # Time-based features
    df['hour'] = (df['timestamp'] // 3600) % 24  # Extract hour from timestamp
    df['minute'] = (df['timestamp'] // 60) % 60  # Extract minute from timestamp
    df['day_of_week'] = (df['timestamp'] // 86400) % 7  # Day of the week
    df['is_market_open'] = ((df['hour'] == 9) & (df['minute'] >= 30)) | ((df['hour'] == 10) & (df['minute'] == 0))
    df['is_market_close'] = ((df['hour'] == 15) & (df['minute'] >= 30)) | ((df['hour'] == 16) & (df['minute'] == 0))
    
    # Price change features
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df['volatility'] = df['log_return'].rolling(window=10).std()
    
    # Calculate RSI for each symbol (group by 'symbol')
    rsi = df.groupby('symbol')['close'].apply(lambda x: RSIIndicator(x, window=14).rsi()).reset_index(level=0, drop=True)
    df['rsi'] = rsi

    df['roc'] = df.groupby('symbol')['close'].pct_change(periods=10)
    atr = df.groupby('symbol').apply(lambda x: AverageTrueRange(x['high'], x['low'], x['close'], window=14).average_true_range()).reset_index(level=0, drop=True)
    if isinstance(atr, pd.Series):
        df['atr'] = atr
    else:
        df['atr'] = atr.stack().reset_index(drop=True)
    df['hist_volatility'] = df['log_return'].rolling(20).std()

    # Target variable: price movement in 10 minutes
    df['future_close'] = df.groupby('symbol')['close'].shift(-10)
    threshold = 0.005  # 0.5% change
    df['target'] = np.where(df['future_close'] > (df['close'] * (1 + threshold)), 1, np.where(df['future_close'] < (df['close'] * (1 - threshold)), 2, 0))
    df['target'] = df['target'].astype(np.int8)

    # Add S&P 500 features to the main DataFrame
    df_sp["sp_return_10m"] = df_sp["close"].pct_change(10)
    df_sp["sp_sma"] = df_sp["close"].rolling(window=10).mean()
    df_sp['sp_log_return'] = np.log(df_sp['close'] / df_sp['close'].shift(1))
    df_sp['sp_volatility'] = df_sp['sp_log_return'].rolling(window=10).std()
    df_sp = df_sp[['timestamp', 'sp_return_10m', 'sp_sma', 'sp_volatility', 'sp_log_return']]
    df = df.merge(df_sp, on='timestamp', how='left')
    
    # Drop rows with NaN values
    df = df.dropna()
    
    return df.astype({  # Convert to optimized dtypes
        'hour': np.int8,
        'minute': np.int8,
        'day_of_week': np.int8,
        'is_market_open': np.int8,
        'is_market_close': np.int8,
        'target': np.int8
    })

In [ ]:
import pandas as pd

# Read the parquet file
file_path = "./data/raw/raw__nasdaq100_2000_to_2025.parquet"
nasdaq100_data = pd.read_parquet(file_path, engine="pyarrow")
file_path_sp = "./data/raw/raw__sp500_2000_to_2025.parquet"
sp500_data = pd.read_parquet(file_path_sp, engine="pyarrow")

# Perform feature engineering
df = feature_engineering(nasdaq100_data, sp500_data)

# Save data to a parquet file
df.to_parquet("./data/enriched/enriched__nasdaq100_2000_to_2025.parquet", engine="pyarrow", index=False)

C:\Users\aolivest\AppData\Local\Temp\ipykernel_34076\2277599990.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['atr'] = df.groupby('symbol').apply(lambda x: AverageTrueRange(x['high'], x['low'], x['close'], window=14).average_true_range()).reset_index(level=0, drop=True)


#### Create small dataset

In [4]:
import pandas as pd
# Read the parquet file
file_path = "./data/raw/raw__nasdaq100_2000_to_2025.parquet"
df = pd.read_parquet(file_path, engine="pyarrow")
file_path_sp = "./data/raw/raw__sp500_2000_to_2025.parquet"
df_sp = pd.read_parquet(file_path_sp, engine="pyarrow")

df_filtered = df[df["timestamp"] > "2023-12-31"]
df_sp = df_sp[df_sp["timestamp"] > "2023-12-31"]

# Ensure 'timestamp' is in datetime format
df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])
df_sp['timestamp'] = pd.to_datetime(df_sp['timestamp'])

# Extract the time from the timestamp
df_filtered['time'] = df_filtered['timestamp'].dt.time
df_sp['time'] = df_sp['timestamp'].dt.time

# Define the start and end times
start_time = pd.to_datetime('09:30:00').time()
end_time = pd.to_datetime('16:00:00').time()

# Filter the rows where the time is between 9:30 and 16:00
df_filtered = df_filtered[(df_filtered['time'] >= start_time) & (df_filtered['time'] <= end_time)]
df_sp = df_sp[(df_sp['time'] >= start_time) & (df_sp['time'] <= end_time)]

# Save data to a parquet file
df_filtered.to_parquet("./data/raw/raw__nasdaq100_2024.parquet", engine="pyarrow", index=False)
df_sp.to_parquet("./data/raw/raw__sp500_2024.parquet", engine="pyarrow", index=False)

C:\Users\aolivest\AppData\Local\Temp\ipykernel_34076\3556077478.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])
C:\Users\aolivest\AppData\Local\Temp\ipykernel_34076\3556077478.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['time'] = df_filtered['timestamp'].dt.time


#### Load Test Data

Single stock:

In [25]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
import pandas as pd
import os

# Keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

stock_client = StockHistoricalDataClient(API_KEY,  SECRET_KEY)

# ------ Get Stock Data ------
stock = "SPY"
request_params = StockBarsRequest(symbol_or_symbols=[stock], timeframe=TimeFrame.Minute, start=datetime(2025, 1, 1), end=datetime(2025, 4, 1))
bars = stock_client.get_stock_bars(request_params)

# Flatten the dictionary into a DataFrame
df = pd.DataFrame(bars[stock])

# Extrahera endast värdena från tuple-paren
df = df.map(lambda x: x[1])  # Tar bara det andra elementet i tuple:n

# Sätt kolumnnamn
df.columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]

# ------ Get SP500 Data ------
stock = "SPY"
request_params = StockBarsRequest(symbol_or_symbols=[stock], timeframe=TimeFrame.Minute, start=datetime(2025, 1, 1), end=datetime(2025, 4, 1))
bars = stock_client.get_stock_bars(request_params)

# Flatten the dictionary into a DataFrame
df_sp = pd.DataFrame(bars[stock])

# Extrahera endast värdena från tuple-paren
df_sp = df_sp.map(lambda x: x[1])  # Tar bara det andra elementet i tuple:n

# Sätt kolumnnamn
df_sp.columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]

# ------ Feature Engineering ----
df = feature_engineering(df, df_sp)

# Save data to a parquet file
df.to_parquet("./data/test/test__spy.parquet", engine="pyarrow", index=False)

C:\Users\aolivest\AppData\Local\Temp\ipykernel_34352\1870289914.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  atr = df.groupby('symbol').apply(lambda x: AverageTrueRange(x['high'], x['low'], x['close'], window=14).average_true_range()).reset_index(level=0, drop=True)


Several stocks:

In [24]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from pytickersymbols import PyTickerSymbols
from datetime import datetime
import pandas as pd
import os

# Keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

# Create client
stock_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Get all stock symbols in NASDAQ 100
stock_data = PyTickerSymbols()
nasdaq_100 = stock_data.get_stocks_by_index('NASDAQ 100')

# Create empty DataFrame
columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]
stock_df = pd.DataFrame(columns=columns)

stock_nr = 0
for stock in nasdaq_100:
    symbol = stock["symbol"]
    stock_nr += 1
    print(f"Processing stock {stock_nr}/100 - {symbol}")

    # Get the stock bars
    request_params = StockBarsRequest(symbol_or_symbols=[symbol], timeframe=TimeFrame.Minute, start=datetime(2025, 1, 1), end=datetime(2025, 4, 1))
    bars = stock_client.get_stock_bars(request_params)
    
    # Check if the symbol is in the data
    if (symbol in bars.data):
        # Clean stock data and add to DataFrame
        df = pd.DataFrame(bars[stock["symbol"]])
        df = df.map(lambda x: x[1]) 
        df.columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]
        if not df.empty:
            stock_df = pd.concat([stock_df, df], ignore_index=True)

# ------ Get SP500 Data ------
stock = "SPY"
request_params = StockBarsRequest(symbol_or_symbols=[stock], timeframe=TimeFrame.Minute, start=datetime(2025, 1, 1), end=datetime(2025, 4, 1))
bars = stock_client.get_stock_bars(request_params)

# Flatten the dictionary into a DataFrame
df_sp = pd.DataFrame(bars[stock])

# Extrahera endast värdena från tuple-paren
df_sp = df_sp.map(lambda x: x[1])  # Tar bara det andra elementet i tuple:n

# Sätt kolumnnamn
df_sp.columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume", "trade_count", "vwap"]

# ------ Feature Engineering ----
df = feature_engineering(stock_df, df_sp)

df = df.sort_values(by="timestamp", ascending=True)

# Save data to a parquet file
df.to_parquet("./data/test/test__nasdaq_100.parquet", engine="pyarrow", index=False)

Processing stock 1/100 - AZN


C:\Users\aolivest\AppData\Local\Temp\ipykernel_34352\3014329166.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stock_df = pd.concat([stock_df, df], ignore_index=True)


Processing stock 2/100 - AAPL
Processing stock 3/100 - CSCO
Processing stock 4/100 - INTC
Processing stock 5/100 - MSFT
Processing stock 6/100 - WBA
Processing stock 7/100 - ATVI
Processing stock 8/100 - ADBE
Processing stock 9/100 - GOOGL
Processing stock 10/100 - AMZN
Processing stock 11/100 - AMGN
Processing stock 12/100 - ADI
Processing stock 13/100 - AMAT
Processing stock 14/100 - ADSK
Processing stock 15/100 - ADP
Processing stock 16/100 - BIDU
Processing stock 17/100 - BIIB
Processing stock 18/100 - CHTR
Processing stock 19/100 - CTSH
Processing stock 20/100 - CMCSA
Processing stock 21/100 - COST
Processing stock 22/100 - CSX
Processing stock 23/100 - DLTR
Processing stock 24/100 - EBAY
Processing stock 25/100 - EA
Processing stock 26/100 - FAST
Processing stock 27/100 - FISV
Processing stock 28/100 - GILD
Processing stock 29/100 - ILMN
Processing stock 30/100 - INTU
Processing stock 31/100 - ISRG
Processing stock 32/100 - JD
Processing stock 33/100 - KHC
Processing stock 34/100

C:\Users\aolivest\AppData\Local\Temp\ipykernel_34352\1870289914.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  atr = df.groupby('symbol').apply(lambda x: AverageTrueRange(x['high'], x['low'], x['close'], window=14).average_true_range()).reset_index(level=0, drop=True)


#### Write Stock Symbols In Dataset to File

In [10]:
import pandas as pd
from pytickersymbols import PyTickerSymbols

# Read the parquet file
# Get all stock symbols in NASDAQ 100
stock_data = PyTickerSymbols()
nasdaq_100 = stock_data.get_stocks_by_index('NASDAQ 100')

stocks = []
for stock in nasdaq_100:
    stocks.append(stock["symbol"])

# Convert the numpy array to a DataFrame and save as a Parquet file
nasdaq_stocks_df = pd.DataFrame(stocks, columns=["symbol"])
nasdaq_stocks_df.to_parquet("../data/enriched/enriched__nasdaq100_stocks.parquet", engine="pyarrow", index=False)